In [1]:
"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/Shareddrives/LIBS/LIBS_library
"""

import numpy as np

# data

In [3]:
from src.visualization import IndexType

INDEX = IndexType.HORIZONTAL_SNAKE
DIM = (266, 500)

ImportError: cannot import name 'IndexType' from 'src.visualization' (c:\Users\Pavel\Desktop\git\libs-map-analysis\src\visualization.py)

In [ ]:
from src.preprocessing import match_wavelengths

s1_wavelengths = np.load(open('data/s1_wavelengths.npy', 'rb'))
s2_wavelengths = np.load(open('data/s2_wavelengths.npy', 'rb'))
s1_spectra = np.load(open('data/s1.npy', 'rb'))
s2_spectra = np.load(open('data/s2.npy', 'rb'))

s1, s2, calibration = match_wavelengths(s1_spectra, s2_spectra, s1_wavelengths, s2_wavelengths)
del s1_spectra, s2_spectra, s1_wavelengths, s2_wavelengths

# layout

In [ ]:
import dash
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)
app.title = 'LIBS Transfer library'